In [12]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from random import randint

# Вывод функции в общем виде
$$
f_0(x) = \dfrac 1 2 (x^TAx) + b^Tx \quad A-неопределенная~невыражденная~матрица
$$
$$
f_0(x) = \dfrac 1 2 \left( \sum\limits_{j=1}^{4}\left( x_j\sum\limits_{i=1}^{4}a_{ij}x_i\right) \right) + \sum\limits_{i=1}^{4}b_ix_i
$$


In [13]:
A = np.array([[ 0, -2,  1, -1],
              [ -2,  1,  0, 1],
              [ 1, 0,  2, 1], 
              [ -1, 1,  1, -1], ])

b = np.array([1, 2, -1, 0])

In [14]:
np.linalg.det(A)

np.float64(21.999999999999996)

$$
A= \begin{pmatrix}
{0}   && {-2} && {1} && {-1} \\ 
{-2}  && {1}  && {0} && {1} \\ 
{1}   && {0}  && {2} && {1} \\
{-1}  && {1}  && {1} && {-1}
\end{pmatrix}
$$

$$
b = \begin{pmatrix}{1}\\ {2}\\ {-1} \\ {0}\end{pmatrix}
$$

$$
f_0(x_1, x_2, x_3, x_4) = 
\dfrac 1 2 \left(
    x_1(-2x_2+x_3-x_4) + x_2(-2x_1+x_2+x_4) + x_3(x_1+2x_3+x_4) + x_4(-x_1+x_2+x_3-x_4)
\right) +
x_1 + 2x_2 - x_3 =
$$


$$
=
\dfrac 1 2 \left( 
    x_2^2 +2x_3^2-x_4^2-4x_1x_2+2x_1x_3-2x_1x_4+2x_2x_4+2x_3x_4
\right) + x_1 + 2x_2 - x_3
$$

$$
g(x)=\| x-x_0 \| - r \le 0 \iff \| x - x_0 \| ^2 - r^2 \le 0
$$

$$
F(x)=f(x) + \lambda g(x)
$$

In [15]:
def f(x1, x2, x3, x4):
    return (x2 ** 2 + 2 * x3 ** 2 - x4 ** 2 - 4 * x1 * x2 + 2 * x1 * x3 - 2 * x1 * x4 + 2 * x2 * x4 + 2 * x3 * x4) / 2 + x1 + 2 * x2 - x3

$$
\nabla F = \begin{cases}
{-2x_2+x_3+x_4+1+2\lambda (x_1-p_1)} \\
{x_2-2x_1+x_4+2+2\lambda (x_2-p_2)} \\
{2x_3+x_1+x_4-1+2\lambda (x_3-p_3)} \\
{-x_4-x_1+x_2+x_3+2\lambda (x_4-p_4)} \\
{(x_1-p_1)^2 + (x_2-p_2)^2 + (x_3-p_3)^2 + (x_4-p_4)^2-r^2}
\end{cases}
$$

In [16]:
def grad_F(x1, x2, x3, x4, lambda_val):
    global x0, r
    return np.array([
        -2 * x2 + x3 - x4 + 1 + 2 * lambda_val * (x1 - x0[0]),
        x2 - 2 * x1 + x4 + 2 +2 * lambda_val * (x2 - x0[1]),
        2 * x3 + x1 + x4 - 1 + 2 * lambda_val * (x3 - x0[2]),
        -x4 - x1 + x2 + x3 + 2 * lambda_val * (x4 - x0[3]), 
        (x1 - x0[0]) ** 2 + (x2 - x0[1]) ** 2 + (x3 - x0[2]) ** 2 + (x4 - x0[3]) ** 2 - r ** 2
    ])

$$
J(x, \lambda) = \begin{pmatrix}
    {2\lambda} && {-2} && {1} && {-1} && {2(x_1-p_1)} \\
    {-2} && {1+2\lambda} && {0} && {1} && {2(x_2-p_2)} \\
    {1} && {0} && {2+2\lambda} && {1} && {2(x_3-p_3)} \\
    {-1} && {1} && {1} && {-1+2\lambda} && {2(x_4-p_4)} \\
    {2(x_1-p_1)} && {2(x_2-p_2)} && {2(x_3-p_3)} && {2(x_4-p_4)} && {0} 
\end{pmatrix}
$$

In [17]:
def Jacob_F(x1, x2, x3, x4, lambda_val):
    global x0
    return np.array([
        [
            2 * lambda_val, -2, 1, -1, 2 * (x1 - x0[0])
        ], 
        [   
            -2, 1 + 2 * lambda_val, 0, 1, 2 * (x2 - x0[1])
        ], 
        [   
            1, 0, 2 + 2 * lambda_val, 1, 2 * (x3 - x0[2])
        ], 
        [
            -1, 1, 1, -1 + 2 * lambda_val, 2 * (x4 - x0[3])
        ], 
        [
            2 * (x1 - x0[0]), 2 * (x2 - x0[1]), 2 * (x3 - x0[2]), 2 * (x4 - x0[3]), 0
        ]
    ])

In [18]:
def МетодНьютона(v0, epsilon):
    global x0, r
    x1, x2, x3, x4 = v0
    lambda_val = 1
    index = 0

    indexes, x1_vals, x2_vals, x3_vals, x4_vals, lambda_vals, func_vals, dF_vals, len_val = [], [], [], [], [], [], [], [], []

    while True:
        indexes.append(index)
        x1_vals.append(x1)
        x2_vals.append(x2)
        x3_vals.append(x3)
        x4_vals.append(x4)
        lambda_vals.append(lambda_val)

        v = np.array([x1, x2, x3, x4, lambda_val])
        dF_val = grad_F(x1, x2, x3, x4, lambda_val)
        norm = sum([dF_val[i] ** 2 for i in range(4)])
        dF_vals.append(norm)

        len_val.append(sum(((v[:-1] - x0) ** 2)))

        f_val = f(x1, x2, x3, x4)
        func_vals.append(f_val)

        J = Jacob_F(x1, x2, x3, x4, lambda_val)

        v = (v.T - np.linalg.inv(J) @ dF_val.T)
        x1, x2, x3, x4, lambda_val = v[0], v[1], v[2], v[3], v[4]

        if norm <= epsilon:
            break
        
        index += 1


    return pd.DataFrame({'x1': x1_vals, 'x2': x2_vals, 
                  'x3': x3_vals, 'x4': x4_vals,
                  'λ': lambda_vals,'f': func_vals, 
                  'dF': dF_vals, 'g<0': len_val}, index=indexes), index - 1, (x1, x2, x3, x4)

In [19]:
x0, r = (3, 4, -5, -6), 1
epsilon = 0.00001

In [20]:
df, index, vec = МетодНьютона((x0[0], x0[1], x0[2] + r, x0[3]), epsilon)
df

,x1,x2,x3,x4,λ,f,dF,g<0
0,3.000000,4.000000,-4.000000,-6.000000,1.000000,3.000000,1.700000e+02,1.000000
1,16.000000,15.000000,-4.000000,-7.000000,0.000000,-355.000000,1.164000e+03,292.000000
2,9.834118,9.163265,-2.721088,-4.581633,0.594846,-121.671049,1.126202e+02,80.569685
3,7.606393,3.760271,-5.743657,2.627327,1.283401,-68.609616,1.806547e+02,96.260124
4,10.642565,9.438827,-4.066030,-4.212216,1.078741,-136.189057,1.525613e+01,92.058115
5,5.822205,10.790600,-0.058208,-15.278643,1.031107,-232.044999,1.484765e+00,164.591608
6,33.102544,22.973802,-11.142674,4.847214,0.829065,-1526.438506,2.319550e+02,1421.562827
7,19.314719,21.114419,-3.134351,-14.582727,0.874386,-665.630975,5.218835e+00,636.217269
8,12.058318,12.548216,-3.117835,-8.448580,0.964021,-224.981189,5.259834e+00,164.663222
9,20.219755,5.088006,-13.322781,21.275428,1.547673,-1072.685115,1.512373e+03,1110.921381


In [21]:
df, index, vec = МетодНьютона((x0[0] + 0.001 * r, x0[1], x0[2], x0[3]), epsilon)
df

,x1,x2,x3,x4,λ,f,dF,g<0
0,3.001000,4.000000,-5.000000,-6.000000,1.000000,15.994000,2.719680e+02,1.000000e-06
1,503.000500,112.000100,-297.000300,676.000700,98001.100000,-860135.749601,3.119588e+16,8.120537e+05
2,253.000922,58.000099,-151.000206,335.000559,49001.019274,-214571.504327,1.949742e+15,2.030139e+05
3,128.001477,31.000197,-78.000270,164.500906,24500.917460,-53405.928947,1.218589e+14,5.075398e+04
4,65.502443,17.500443,-41.500526,79.251914,12250.743655,-13227.278523,7.616185e+12,1.268899e+04
5,34.254302,10.750959,-23.251101,36.629088,6125.410980,-3238.988539,4.760122e+11,3.172749e+03
6,18.632985,7.377004,-14.127281,15.321013,3062.253303,-770.104205,2.975093e+10,7.936872e+02
7,10.827830,5.691599,-9.567156,4.673649,1529.693417,-166.981038,1.859470e+09,1.989221e+02
8,6.936243,4.852043,-7.290643,-0.636718,761.464369,-23.257536,1.162217e+08,5.023182e+01
9,5.012300,4.438552,-6.159316,-3.265524,373.554073,9.124377,7.260722e+06,1.306305e+01


In [22]:
df, index, vec = МетодНьютона((x0[0], x0[1] - 100 * r, x0[2], x0[3]), epsilon)
df

,x1,x2,x3,x4,λ,f,dF,g<0
0,3.000000,-96.000000,-5.000000,-6.000000,1.000000,5616.000000,1.410720e+05,10000.000000
1,73.005000,-46.005000,-63.005000,170.015000,0.399950,-38208.460275,6.012435e+04,41747.060300
2,-6.520595,-47.296998,-23.639257,95.663170,0.623677,-9600.752277,2.683642e+03,13404.845784
3,-26.278066,-38.081967,-4.613806,36.027984,0.623785,-2472.126577,2.048847e-04,4394.597692
4,-11.074661,-16.720790,-3.036673,15.723386,0.591720,-489.325321,4.532941e+00,1103.207419
5,-3.451208,-5.955116,-2.080680,5.475913,0.510688,-65.241270,7.352458e+00,280.941421
6,0.483445,-0.325557,-1.226388,0.157078,0.332218,2.005576,9.707568e+00,77.193247
7,3.206799,3.637930,0.499970,-3.190853,-0.154089,-12.940768,3.529892e+01,38.314843
8,0.546737,-0.334704,-4.048360,-1.243646,0.762908,23.881125,1.592182e+02,48.336674
9,2.530374,2.592763,-1.375348,-3.063527,0.107880,-2.927739,3.940808e+01,23.961839
